In [1]:
from translating_the_law.downloading.get_data import get_data

In [ ]:
basic_data = get_data(num = 20)

Basic data is a list and each item inside the list is a dictionary with these keys

In [ ]:
basic_data[0].keys()

and then inside those keys dictionaries with these keys

In [ ]:
txt_1 = basic_data[0]["judgement"]['body']

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts
import numpy as np
import pandas as pd


#### Copy dataset


In [ ]:
judg_data = [basic_data[i]["judgement"]['body'] for i in range(len(basic_data))]
dfc = pd.DataFrame(judg_data).copy()

# Preprocessing

#### Clean text

In [ ]:
import string
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize



In [ ]:
dfc = dfc.rename(columns={0 : 'text'})

In [ ]:
import string

def remove_punctuation(text):
    for punctuation in string.punctuation: 
        text = text.replace(punctuation, ' ') 
    return text

dfc['clean_text'] = dfc.text.apply(remove_punctuation)

In [ ]:
def lowercase (text): 
    lowercased = text.lower() 
    return lowercased

dfc['clean_text'] = dfc.clean_text.apply(lowercase)



In [ ]:
import re
def cleaner(text, **kwargs):
  """params is a list of things to remove: codec, acronyms, numbers, brackets"""
  if 'codec' in kwargs['params']:
    text_encoded = text.encode('ascii', errors = 'ignore')
    text_decode = text_encoded.decode()
    clean_text = " ".join([word for word in text_decode.split()])
    text = clean_text
  if 'numbers' in kwargs['params']:
    pattern = r'[0-9]'
    text = re.sub(pattern, '', text)
  if 'brackets' in kwargs['params']: 
    text = re.sub('\(.*?\)', '', text)
    text = re.sub('\[.*?\]', '', text)
  if 'acronyms' in kwargs['params']:
    text = text.split()
    clean_text = []
    for word in text: 
      if any(l.islower() for l in word):
        clean_text.append(word)
    text = ' '.join(clean_text)
  return text


In [ ]:

string_test = dfc['clean_text'][0]
params = ['codec', 'numbers', 'acronyms', 'brackets']
cleaner(string_test, params=params)

In [ ]:
def remove_numbers (text):
    words_only = ''.join([i for i in text if not i.isdigit()])
    return words_only

dfc['clean_text'] = dfc.clean_text.apply(remove_numbers)


#### get a list of lists of words and lemmatize

In [ ]:
from nltk.corpus import stopwords 
from nltk import word_tokenize

stop_words = set(stopwords.words('english')) 

# Create function
def remove_stopwords (text):
    tokenized = word_tokenize(text)
    without_stopwords = [word for word in tokenized if not word in stop_words]
    return without_stopwords

dfc['clean_text'] = dfc.clean_text.apply(remove_stopwords)

dfc.head()


In [ ]:
from nltk.stem import WordNetLemmatizer

def lemma(text):
    lemmatizer = WordNetLemmatizer() # Initiate lemmatizer
    lemmatized = [lemmatizer.lemmatize(word) for word in text] # Lemmatize
    return lemmatized

dfc['clean_text'] = dfc.clean_text.apply(lemma)

dfc.head()

# LDA : extract topic

In [ ]:
dfc['clean_text'] = dfc['clean_text'].astype('str')
dfc['clean_text'][0]

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

data_vectorized = vectorizer.fit_transform(dfc['clean_text'])


In [ ]:

lda_model = LatentDirichletAllocation(n_components=10, learning_method='online')

lda_vectors = lda_model.fit_transform(data_vectorized)

In [ ]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])
        

print_topics(lda_model, vectorizer)

# Initialize Model

In [ ]:
import gensim
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
def tagged_document(list_of_list_of_words):
   for i, list_of_words in enumerate(list_of_list_of_words):
      yield TaggedDocument(list_of_words, [i])
data_for_training = list(tagged_document(dfc['clean_text']))


#### Once trained we now need to initialise the model.

In [ ]:
model = Doc2Vec(documents = data_for_training)

#### Build Vocabulary

In [ ]:
model.build_vocab(data_for_training)

#### Train the Doc2Vec 

In [ ]:
model.train(data_for_training, total_examples=model.corpus_count, epochs=model.epochs)

## Analysing the Output

In [ ]:
vec1 = model.infer_vector(test_1)

In [ ]:
len(vec1)

In [ ]:
model